# CASO PRÁCTICO

## Ejercicio 1

Con el csv de acciones de Apple (AAPL.csv), realiza las siguientes visualizaciones:
* **Matplotlib**: Obtén un diagrama de barras de las 5 primeras fechas del dataset para los precios de apertura y cierre.
* **Ggplot**: Realiza una nube de puntos con el volumen de acción y el cierre ajustado, muestra además la línea de ajuste (regresión lineal), entre ambas variables, utiliza un gradiente de color mediante el precio de cierre ajustado.
* **Seaborn 1**: Realiza un mapa de calor mediante la correlación del dataframe
* **Seaborn 2**: Obtén un histograma desactivando la función de densidad
* **Bokeh**: Realiza una gráfica de toda la serie temporal sobre los precios más altos y más bajos, para que se diferencien más las líneas resta 40 puntos a todos los precios más bajos.

### SOLUCIÓN

Importamos todas las librerías y realizamos una pequeña descripción de los datos.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from plotnine import *
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

In [ ]:
df = pd.read_csv('AAPL.csv')

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
print("*CANTIDAD de datos nulos por columna en el dataframe")
print(df.isnull().sum())
print("----------------------------------")
print("*PORCENTAJE de datos nulos por columna en el dataframe")
print(df.isnull().sum()/len(df)*100)

## Matplotlib

Obtenemos las 5 primeras posiciones de las fechas, precios de apertura y cierre.

En la gráfica, debemos tener en cuenta que los valores de la fecha, es necesario que se pasen como tuplas.

In [ ]:
df_ten = df[['Date', 'Open', 'Close']].head(5)

In [ ]:
n_groups = len(df_ten)

# create plot
fig, ax = plt.subplots()
index = np.arange(n_groups)


min_ = plt.bar(index , df_ten['Close'], color='lightblue', 
                 label='Cierre')

max_ = plt.bar(index, df_ten['Open'], color='tomato', 
                 label='Apertura')

plt.xlabel('Fechas')
plt.ylabel('Valores')
plt.title('Valores de la acción')
plt.xticks(index ,tuple(df['Date']))
plt.legend()

plt.tight_layout()
plt.show()

## Ggplot

Simplemente, mostramos dos capas diferentes una con el scatter plot y otra con el ajuste lineal (Evidentemente no hay relación lineal entre ambas variables).

In [ ]:
(
    ggplot(df) +
     geom_point(aes(x='Volume', y = 'Adj Close', fill = 'Adj Close'),
               alpha=0.5,
               shape='o',
               size = 2,
               stroke = 0.2) 
    + geom_smooth(
        aes(x='Volume', y = 'Adj Close')
    )
    + labs(title='Volumen y cierre ajustado', 
         x = 'Volumen de acciones',
         y = 'Cierre ajustado')
)


## Seaborn 1

Utilizamos la función corr de pandas para ayudarnos.

In [ ]:
sns.heatmap(df.corr(), cmap='Greens')

## Seaborn 2

Para desactivar la función de densidad, pasamos False al parámetro kde.

In [ ]:
sns.distplot(a=df.Volume, color='blue', kde=False)

## Bokeh

Aquí es importante utilizar el formato de fechas adecuado, si simplemente pasamos en el eje x la columna 'Date' no aparecerá nada y es que, debemos cambiar el tipo de las fechas a numpy datetime64.

Finalmente, tenemos que especificar en figure al parámetro x_axis_type que nuestro eje x es formato **datetime**

In [ ]:
output_notebook()

In [ ]:
correct_dates = np.array(df['Date'], dtype=np.datetime64)

In [ ]:
p = figure(plot_width=800, plot_height=400, title = 'Acciones de Apple', x_axis_type="datetime")


p.line(correct_dates, df['High'], 
       color = 'blue', line_width=1, line_alpha = 0.5)
p.line(correct_dates, df['Low']-40, 
       color = 'green', line_width=1, line_alpha = 0.5)

p.xaxis.axis_label = 'Fechas (array)'
p.yaxis.axis_label = 'Valor de acción'

show(p)

Otra forma con elementos dinámicos

In [ ]:
from bokeh.models import ColumnDataSource, HoverTool

df['New_Date'] = correct_dates
df['Low'] = df['Low']-40
source = ColumnDataSource(df)

In [ ]:
source.column_names

In [ ]:
p = figure(plot_width=800, plot_height=400, title = 'Acciones de Apple', x_axis_type="datetime")


p.line('New_Date', 'High', source = source,
       color = 'purple', line_width=1, line_alpha = 0.5)
p.line('New_Date', 'Low', source = source,
       color = 'orange', line_width=1, line_alpha = 1)

p.xaxis.axis_label = 'Fechas (array)'
p.yaxis.axis_label = 'Valor de acción'

hover = HoverTool()
hover.tooltips=[
    ('Max', '@High'),
    ('Min', '@Low'),
    ('Apertura', '@Open'),
    ('Cierre', '@Close'),
    ('Fecha', '@Date')
]

p.add_tools(hover)


show(p)